In [3]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [4]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_vci.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-4-0aa77f748b21>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
VCI,20210428,62.5,65.5,62.5,64.0,1834700,62.5,65.5,62.5,64.0,0,12800,10400,2021-04-28,17,2021,2021-17
VCI,20210427,61.0,62.5,60.8,62.3,843500,61.0,62.5,60.8,62.3,0,43800,200200,2021-04-27,17,2021,2021-17
VCI,20210426,62.8,63.0,60.8,61.0,1024700,62.8,63.0,60.8,61.0,0,68100,71200,2021-04-26,17,2021,2021-17


In [5]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [19]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='VCI '))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [7]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [8]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
VCI,59.0,2017-07-11,838360,2017-28,2017
VCI,60.1,2017-07-10,2677240,2017-28,2017
VCI,57.6,2017-07-07,114260,2017-27,2017


In [9]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
VCI,62.3,2021-04-27,843500,2021-17,2021,-2.656250
VCI,61.0,2021-04-26,1024700,2021-17,2021,-2.086677
VCI,62.2,2021-04-23,1100100,2021-16,2021,1.967213


In [10]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [11]:
df['Daily_return'].describe()

count    951.000000
mean       0.026046
std        2.777738
min       -6.534091
25%       -1.357787
50%        0.000000
75%        1.148483
max       35.714286
Name: Daily_return, dtype: float64

In [12]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [13]:
df['<Close>'].describe()

count    951.000000
mean      49.019085
std       22.484325
min       14.200000
25%       31.900000
50%       43.500000
75%       61.100000
max      118.000000
Name: <Close>, dtype: float64

In [14]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [15]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [16]:
df['Volumn'].head(30).describe()

count    30.000000
mean      1.477090
std       0.348590
min       0.843500
25%       1.177975
50%       1.496550
75%       1.721300
max       2.150000
Name: Volumn, dtype: float64

In [17]:
df['Volumn'].head(10).describe()

count    10.000000
mean      1.330420
std       0.379352
min       0.843500
25%       1.114625
50%       1.227250
75%       1.450325
max       2.122800
Name: Volumn, dtype: float64

In [18]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
VCI,62.3,2021-04-27,843500,2021-17,2021,-2.656250,0.8435
VCI,61.0,2021-04-26,1024700,2021-17,2021,-2.086677,1.0247
VCI,62.2,2021-04-23,1100100,2021-16,2021,1.967213,1.1001
